In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [3]:
df = pd.read_csv('../data/processed/cleaned_essays.csv')

In [4]:
# Encode target variable
le = LabelEncoder()
df['topic_encoded'] = le.fit_transform(df['prompt_name'])
num_classes = len(le.classes_)

In [5]:
# Save label encoder classes
import pickle
with open('../model/label_encoder.pkl', 'wb') as f:
    pickle.dump(le.classes_, f)

In [6]:
# Tokenize text data
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['clean_text'])

X = tokenizer.texts_to_sequences(df['clean_text'])
X = pad_sequences(X, maxlen=max_len)

# Save tokenizer
with open('../model/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

y = to_categorical(df['topic_encoded'], num_classes=num_classes)

In [7]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Build LSTM Model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

c:\Users\Dell\anaconda3\envs\myenv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:
# Train Model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
619/619 ━━━━━━━━━━━━━━━━━━━━ 220s 334ms/step - accuracy: 0.6202 - loss: 1.0624 - val_accuracy: 0.9179 - val_loss: 0.2462
Epoch 2/5
619/619 ━━━━━━━━━━━━━━━━━━━━ 203s 328ms/step - accuracy: 0.9253 - loss: 0.2398 - val_accuracy: 0.9842 - val_loss: 0.0862
Epoch 3/5
619/619 ━━━━━━━━━━━━━━━━━━━━ 198s 320ms/step - accuracy: 0.9616 - loss: 0.1344 - val_accuracy: 0.9860 - val_loss: 0.0713
Epoch 4/5
619/619 ━━━━━━━━━━━━━━━━━━━━ 196s 317ms/step - accuracy: 0.9852 - loss: 0.0624 - val_accuracy: 0.9917 - val_loss: 0.0437
Epoch 5/5
619/619 ━━━━━━━━━━━━━━━━━━━━ 200s 323ms/step - accuracy: 0.9827 - loss: 0.0731 - val_accuracy: 0.9735 - val_loss: 0.1203


In [10]:
# Save Model
model.save('../model/topic_detection_model.h5')

print("✅ Model trained and saved.")


✅ Model trained and saved.


In [11]:
# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=10000)
X_tfidf = tfidf.fit_transform(df['clean_text'])

# Save tfidf vectorizer
with open('../model/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

In [12]:
# Train/test split
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(
    X_tfidf, df['topic_encoded'], test_size=0.2, random_state=42
)

In [13]:
# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_ml, y_train_ml)
lr_preds = lr.predict(X_test_ml)

print("\n📊 Logistic Regression Results:")
print("Accuracy:", accuracy_score(y_test_ml, lr_preds))
print(classification_report(y_test_ml, lr_preds, target_names=le.classes_))


📊 Logistic Regression Results:
Accuracy: 0.9987868985038415
                                  precision    recall  f1-score   support

   "A Cowboy Who Rode the Waves"       1.00      1.00      1.00       410
                 Car-free cities       1.00      0.99      1.00       421
Does the electoral college work?       1.00      1.00      1.00       404
                 Driverless cars       1.00      1.00      1.00      1226
                 Exploring Venus       1.00      1.00      1.00       932
     Facial action coding system       1.00      1.00      1.00       946
                The Face on Mars       1.00      1.00      1.00       607

                        accuracy                           1.00      4946
                       macro avg       1.00      1.00      1.00      4946
                    weighted avg       1.00      1.00      1.00      4946



In [14]:
# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_ml, y_train_ml)
rf_preds = rf.predict(X_test_ml)

print("\n📊 Random Forest Results:")
print("Accuracy:", accuracy_score(y_test_ml, rf_preds))
print(classification_report(y_test_ml, rf_preds, target_names=le.classes_))


📊 Random Forest Results:
Accuracy: 0.9977759805903761
                                  precision    recall  f1-score   support

   "A Cowboy Who Rode the Waves"       1.00      1.00      1.00       410
                 Car-free cities       1.00      0.98      0.99       421
Does the electoral college work?       1.00      1.00      1.00       404
                 Driverless cars       0.99      1.00      1.00      1226
                 Exploring Venus       1.00      1.00      1.00       932
     Facial action coding system       1.00      1.00      1.00       946
                The Face on Mars       1.00      1.00      1.00       607

                        accuracy                           1.00      4946
                       macro avg       1.00      1.00      1.00      4946
                    weighted avg       1.00      1.00      1.00      4946



In [15]:
# Save Logistic Regression model
with open('../model/logistic_regression_model.pkl', 'wb') as f:
    pickle.dump(lr, f)

# Save Random Forest model
with open('../model/random_forest_model.pkl', 'wb') as f:
    pickle.dump(rf, f)

print("✅ Logistic Regression and Random Forest models saved.")


✅ Logistic Regression and Random Forest models saved.


In [16]:
# Evaluate LSTM Model
lstm_loss, lstm_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ LSTM Model Accuracy: {lstm_accuracy:.4f}")

# Logistic Regression Accuracy
lr_accuracy = accuracy_score(y_test_ml, lr_preds)
print(f"✅ Logistic Regression Accuracy: {lr_accuracy:.4f}")

# 3. Random Forest Accuracy
rf_accuracy = accuracy_score(y_test_ml, rf_preds)
print(f"✅ Random Forest Accuracy: {rf_accuracy:.4f}")

# Comparison table
results = pd.DataFrame({
    'Model': ['LSTM', 'Logistic Regression', 'Random Forest'],
    'Accuracy': [lstm_accuracy, lr_accuracy, rf_accuracy]
})

print("\n📊 Model Comparison:")
display(results)



✅ LSTM Model Accuracy: 0.9735
✅ Logistic Regression Accuracy: 0.9988
✅ Random Forest Accuracy: 0.9978

📊 Model Comparison:


Model  Accuracy
0                 LSTM  0.973514
1  Logistic Regression  0.998787
2        Random Forest  0.997776